In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import autokeras as ak
import absl.logging

In [2]:
log_dir='.\\logs'
absl.logging.set_verbosity(absl.logging.ERROR)

In [3]:
dataset = pd.read_csv('./line.csv')
dataset.drop(dataset.index[dataset[(dataset.values == 'X') | \
    (dataset.values == 'V')].index], inplace=True)
dataset

,date,hours,temp,dpTemp,RH,WDIR,VIZ,Label
0,2020/1/1,1,11.4,6.7,73,60,10.51,1
1,2020/1/1,2,11.3,6.8,74,50,9.94,1
2,2020/1/1,3,11.4,6.9,74,30,9.89,1
3,2020/1/1,4,11.3,6.8,74,40,10.41,1
4,2020/1/1,5,11.5,7.0,74,50,11.99,1
...,...,...,...,...,...,...,...,...
95,2020/1/4,24,17.0,13.7,81,60,7.47,1
96,2020/1/5,1,16.9,13.6,81,60,5.39,1
97,2020/1/5,2,16.8,13.6,81,60,5.37,1
98,2020/1/5,3,16.5,13.4,82,60,6.36,1


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    100 non-null    object 
 1   hours   100 non-null    int64  
 2   temp    100 non-null    float64
 3   dpTemp  100 non-null    float64
 4   RH      100 non-null    int64  
 5   WDIR    100 non-null    int64  
 6   VIZ     100 non-null    float64
 7   Label   100 non-null    int64  
dtypes: float64(3), int64(4), object(1)
memory usage: 7.0+ KB


In [4]:
val_split = int(len(dataset) * 0.7)
data_train = dataset[:val_split]
validation_data = dataset[val_split:]

In [5]:
data_x = data_train[
    [
        "temp",
        "dpTemp",
        "RH",
        "WDIR"
    ]
].astype("float64")

data_x_val = validation_data[
    [
        "temp",
        "dpTemp",
        "RH",
        "WDIR"
    ]
].astype("float64")

data_x_test = dataset[
    [
        "temp",
        "dpTemp",
        "RH",
        "WDIR"
    ]
].astype("float64")


In [6]:
print(data_x)
print(data_x_val)
print(data_x_test)

    temp  dpTemp    RH  WDIR
0   11.4     6.7  73.0  60.0
1   11.3     6.8  74.0  50.0
2   11.4     6.9  74.0  30.0
3   11.3     6.8  74.0  40.0
4   11.5     7.0  74.0  50.0
..   ...     ...   ...   ...
65  17.2    13.0  76.0  90.0
66  16.8    13.2  79.0  80.0
67  16.5    13.0  80.0  70.0
68  15.9    12.8  82.0  80.0
69  15.6    12.7  83.0  80.0

[70 rows x 4 columns]
    temp  dpTemp    RH   WDIR
70  15.6    12.4  81.0   60.0
71  15.4    12.6  83.0   50.0
72  15.3    13.0  86.0   30.0
73  15.5    13.3  87.0   10.0
74  15.9    14.0  88.0  360.0
75  15.8    13.7  87.0  360.0
76  15.5    13.6  88.0   20.0
77  15.5    13.7  89.0   10.0
78  15.4    13.5  88.0   10.0
79  16.3    15.0  92.0   10.0
80  18.3    15.1  82.0   10.0
81  19.4    15.1  76.0   20.0
82  20.7    14.6  68.0   60.0
83  21.6    14.7  65.0   40.0
84  22.6    14.9  62.0   70.0
85  23.1    15.1  61.0   60.0
86  22.4    15.2  64.0  110.0
87  21.0    14.6  67.0  100.0
88  19.0    13.8  72.0   90.0
89  18.0    13.7  76.0   80.0

In [7]:
data_y = data_train["VIZ"].astype("float64")

data_y_val = validation_data["VIZ"].astype("float64")

In [8]:
print(data_y)
print(data_y_val)

0     10.51
1      9.94
2      9.89
3     10.41
4     11.99
      ...  
65    26.55
66    22.69
67    21.71
68    19.01
69    17.95
Name: VIZ, Length: 70, dtype: float64
70    21.81
71    18.00
72    11.75
73     8.12
74     5.49
75     5.81
76     4.30
77     4.53
78     3.97
79     4.36
80     6.11
81     6.71
82    10.78
83    14.83
84    17.93
85    15.31
86    17.55
87    17.97
88    20.27
89    22.08
90    22.81
91    16.69
92    13.81
93    10.86
94     9.22
95     7.47
96     5.39
97     5.37
98     6.36
99     5.81
Name: VIZ, dtype: float64


In [9]:
print(data_x.shape)
print(data_y.shape)
print(data_x_val.shape)
print(data_y_val.shape)
print(data_x_test.shape)

(70, 4)
(70,)
(30, 4)
(30,)
(100, 4)


In [10]:
predict_from = 1    # default 每次加1
predict_until = 10
lookback = 23
Fog = ak.TimeseriesForecaster(
    lookback=lookback,
    predict_until = predict_until,
    max_trials = 100,
)

In [11]:
Fog.fit(
    x = data_x, y = data_y,
    validation_data = (data_x_val, data_y_val),
    epochs = 100
)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
True              |?                 |timeseries_block_1/rnn_block_1/bidirectional
lstm              |?                 |timeseries_block_1/rnn_block_1/layer_type
2                 |?                 |timeseries_block_1/rnn_block_1/num_layers
0                 |?                 |regression_head_1/dropout
adam              |?                 |optimizer
1e-05             |?                 |learning_rate

Epoch 1/100
      1/Unknown - 5s 5s/step - loss: 374.3822 - mean_squared_error: 374.3822

InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/mean_squared_error/BroadcastGradientArgs' defined at (most recent call last):
    File "c:\ProgramData\Anaconda3\envs\tfenv\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\ProgramData\Anaconda3\envs\tfenv\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\ProgramData\Anaconda3\envs\tfenv\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "c:\ProgramData\Anaconda3\envs\tfenv\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
      app.start()
    File "c:\ProgramData\Anaconda3\envs\tfenv\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
      self.io_loop.start()
    File "c:\ProgramData\Anaconda3\envs\tfenv\lib\site-packages\tornado\platform\asyncio.py", line 149, in start
      self.asyncio_loop.run_forever()
    File "c:\Pro